## 1. Feature engineering
## 2. Proponer las 3 arquitecturas
## 3. Seleccionar la mejor
## 4. Crear version con feature extractor

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

# Medicion de metricas

In [18]:
def calculate_recall(y_pred, y_prueba):
    recall = recall_score(y_prueba, y_pred, average='macro')
    return recall

def calculate_accuracy(y_pred, y_prueba):
    correct = np.sum(np.array(y_pred) == np.array(y_prueba))
    total = len(y_prueba)
    accuracy = correct / total
    return accuracy

def calculate_precision(y_pred, y_prueba):
    precision = precision_score(y_prueba, y_pred, average='macro', zero_division=1)
    return precision

def calculate_f1_score(y_pred, y_prueba):
    f1 = f1_score(y_prueba, y_pred, average='macro')
    return f1

def calculate_metrics(y_pred, y_prueba, start_time, end_time):
    recall = calculate_recall(y_pred, y_prueba)
    accuracy = calculate_accuracy(y_pred, y_prueba)
    precision = calculate_precision(y_pred, y_prueba)
    f1 = calculate_f1_score(y_pred, y_prueba)
    training_time = end_time - start_time
    
    print(pd.DataFrame({
    'Metric': ['Recall', 'Precision', 'Accuracy', 'F1', 'Training Time'],
    'Score': [recall, accuracy, precision, f1, training_time]
    }))

# Carga del set de datos

In [39]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Rutas de las carpetas que contienen las imágenes
covid_folder = 'dataset/images/COVID'
lung_opacity_folder = 'dataset/images/Lung_Opacity'
normal_folder = 'dataset/images/Normal'
viral_pneumonia_folder = 'dataset/images/Viral Pneumonia'

# Lista para almacenar las imágenes y las etiquetas correspondientes
X = []
y = []

# Cargar imágenes de pulmones con COVID
for filename in os.listdir(covid_folder):
    if filename.endswith('.png'):
        img = cv2.imread(os.path.join(covid_folder, filename), cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(0)  # Etiqueta 0 para COVID

# Cargar imágenes de pulmones con opacidad
for filename in os.listdir(lung_opacity_folder):
    if filename.endswith('.png'):
        img = cv2.imread(os.path.join(lung_opacity_folder, filename), cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(1)  # Etiqueta 1 para Opacidad pulmonar

# Cargar imágenes de pulmones sanos
for filename in os.listdir(normal_folder):
    if filename.endswith('.png'):
        img = cv2.imread(os.path.join(normal_folder, filename), cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(2)  # Etiqueta 2 para pulmones sanos

# Cargar imágenes de pulmones con neumonía viral
for filename in os.listdir(viral_pneumonia_folder):
    if filename.endswith('.png'):
        img = cv2.imread(os.path.join(viral_pneumonia_folder, filename), cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(3)  # Etiqueta 3 para Neumonía viral

# Convertir las listas en matrices numpy
X = np.array(X)
y = np.array(y)

In [40]:
# Obtener la cantidad de imágenes en cada clase
cantidad_covid = len(os.listdir(covid_folder))
cantidad_opacidad_pulmonar = len(os.listdir(lung_opacity_folder))
cantidad_pulmones_sanos = len(os.listdir(normal_folder))
cantidad_neumonia_viral = len(os.listdir(viral_pneumonia_folder))

# Imprimir la cantidad de imágenes en cada clase
print("Cantidad de imágenes por clase:\n")
print("COVID:", cantidad_covid)
print("Opacidad pulmonar:", cantidad_opacidad_pulmonar)
print("Pulmones sanos:", cantidad_pulmones_sanos)
print("Neumonía viral:", cantidad_neumonia_viral)

# Aplanar las imágenes en un vector unidimensional
X = X.reshape(X.shape[0], -1)

# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar el tamaño de las matrices
print("Tamaño de X_entrenamiento:", X_entrenamiento.shape)
print("Tamaño de X_prueba:", X_prueba.shape)
print("Tamaño de y_entrenamiento:", y_entrenamiento.shape)
print("Tamaño de y_prueba:", y_prueba.shape)

Cantidad de imágenes por clase:

COVID: 3616
Opacidad pulmonar: 6012
Pulmones sanos: 10192
Neumonía viral: 1345
Tamaño de X_entrenamiento: (16932, 89401)
Tamaño de X_prueba: (4233, 89401)
Tamaño de y_entrenamiento: (16932,)
Tamaño de y_prueba: (4233,)


# Feature engineering

In [4]:
from sklearn.preprocessing import StandardScaler

# Preprocesamiento de las imágenes
preprocessed_images = []
for image in X:
    # Redimensionar la imagen a un tamaño adecuado
    resized_image = cv2.resize(image, (64, 64))  # Ajusta el tamaño según tus necesidades

    # Normalizar la imagen
    normalized_image = resized_image.astype(float) / 255.0  # Normalización en el rango [0, 1]

    preprocessed_images.append(normalized_image)

# Convertir las imágenes preprocesadas en una matriz numpy
X_preprocessed = np.array(preprocessed_images)

# Aplanar las imágenes preprocesadas en un vector unidimensional
X_preprocessed = X_preprocessed.reshape(X_preprocessed.shape[0], -1)

# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Normalizar las características utilizando un escalador estándar
scaler = StandardScaler()
X_entrenamiento = scaler.fit_transform(X_entrenamiento)
X_prueba = scaler.transform(X_prueba)

In [5]:
# Verificar el tamaño de las matrices
print("Tamaño de X_entrenamiento:", X_entrenamiento.shape)
print("Tamaño de X_prueba:", X_prueba.shape)
print("Tamaño de y_entrenamiento:", y_entrenamiento.shape)
print("Tamaño de y_prueba:", y_prueba.shape)

Tamaño de X_entrenamiento: (16932, 4096)
Tamaño de X_prueba: (4233, 4096)
Tamaño de y_entrenamiento: (16932,)
Tamaño de y_prueba: (4233,)


# Sklearn

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import time

In [34]:
mlp = MLPClassifier(activation='logistic', alpha=0.01, hidden_layer_sizes=(10,10), max_iter=1000, random_state=42, solver='sgd')

start_time = time.time()
mlp.fit(X_entrenamiento, y_entrenamiento)
end_time = time.time()

y_pred = mlp.predict(X_prueba)

print("Predicciones: ", y_pred)
print("Valor real  : ",y_prueba)

accuracy = accuracy_score(y_prueba, y_pred)
print("Precisión: {:.2f}".format(accuracy))

Predicciones:  [2 2 2 ... 2 2 2]
Valor real  :  [3 2 0 ... 1 1 0]
Precisión: 0.48


In [19]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric       Score
0         Recall    0.636822
1      Precision    0.851878
2       Accuracy    0.738176
3             F1    0.664353
4  Training Time  422.616639


# Prueba sin feature extractor

In [21]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# Ruta de la carpeta que contiene las imágenes de pulmones con COVID
covid_folder = 'dataset\images\COVID_target'
# Ruta de la carpeta que contiene las imágenes de pulmones sanos
healthy_folder = 'dataset\images\Other'

# Lista para almacenar las imágenes y las etiquetas correspondientes
X = []
y = []

# Cargar imágenes de pulmones con COVID
for filename in os.listdir(covid_folder):
    if filename.endswith('.png'):
        img = cv2.imread(os.path.join(covid_folder, filename), cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(1)  # Etiqueta 1 para COVID

# Cargar imágenes de pulmones sanos
for filename in os.listdir(healthy_folder):
    if filename.endswith('.png'):
        img = cv2.imread(os.path.join(healthy_folder, filename), cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(0)  # Etiqueta 0 para pulmones sanos

# Convertir las listas en matrices numpy
X = np.array(X)
y = np.array(y)

# Aplanar las imágenes en un vector unidimensional
X = X.reshape(X.shape[0], -1)

# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
# Verificar el tamaño de las matrices
print("Tamaño de X_entrenamiento:", X_entrenamiento.shape)
print("Tamaño de X_prueba:", X_prueba.shape)
print("Tamaño de y_entrenamiento:", y_entrenamiento.shape)
print("Tamaño de y_prueba:", y_prueba.shape)

Tamaño de X_entrenamiento: (16932, 89401)
Tamaño de X_prueba: (4233, 89401)
Tamaño de y_entrenamiento: (16932,)
Tamaño de y_prueba: (4233,)


In [23]:
# Verificar el tamaño de las matrices
print("X_entrenamiento:", X_entrenamiento[0:5])
print("X_prueba:", X_prueba[0:5])
print("y_entrenamiento:", y_entrenamiento[0:5])
print("y_prueba:", y_prueba[0:5])

X_entrenamiento: [[158 157 151 ...   1   1   1]
 [ 55  70  75 ...   0   0   0]
 [  5   7   8 ...  48  52  55]
 [  2   3   3 ...  17  14  11]
 [237 237 238 ... 239 239 240]]
X_prueba: [[203 201 198 ...  25  28  29]
 [ 49  59  66 ...   2  15  36]
 [  7  11  14 ...  26  32  45]
 [ 25  27  29 ... 215 216 216]
 [182 176 156 ...   0   0   0]]
y_entrenamiento: [0 0 0 0 0]
y_prueba: [0 0 1 0 0]


# Estandarización

In [14]:
# from sklearn.preprocessing import StandardScaler

# # Preprocesamiento de las imágenes
# preprocessed_images = []
# for image in X:
#     # Normalizar la imagen
#     normalized_image = resized_image.astype(float) / 255.0  # Normalización en el rango [0, 1]

#     preprocessed_images.append(normalized_image)

# # Convertir las imágenes preprocesadas en una matriz numpy
# X_preprocessed = np.array(preprocessed_images)

# # Aplanar las imágenes preprocesadas en un vector unidimensional
# X_preprocessed = X_preprocessed.reshape(X_preprocessed.shape[0], -1)

# # Dividir el conjunto de datos en entrenamiento y prueba
# X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# # Normalizar las características utilizando un escalador estándar
# scaler = StandardScaler()
# X_entrenamiento = scaler.fit_transform(X_entrenamiento)
# X_prueba = scaler.transform(X_prueba)

In [24]:
# Verificar el tamaño de las matrices
print("Tamaño de X_entrenamiento:", X_entrenamiento.shape)
print("Tamaño de X_prueba:", X_prueba.shape)
print("Tamaño de y_entrenamiento:", y_entrenamiento.shape)
print("Tamaño de y_prueba:", y_prueba.shape)

Tamaño de X_entrenamiento: (16932, 89401)
Tamaño de X_prueba: (4233, 89401)
Tamaño de y_entrenamiento: (16932,)
Tamaño de y_prueba: (4233,)


In [25]:
# Verificar el tamaño de las matrices
print("X_entrenamiento:", X_entrenamiento[0:5])
print("X_prueba:", X_prueba[0:5])
print("y_entrenamiento:", y_entrenamiento[0:5])
print("y_prueba:", y_prueba[0:5])

X_entrenamiento: [[158 157 151 ...   1   1   1]
 [ 55  70  75 ...   0   0   0]
 [  5   7   8 ...  48  52  55]
 [  2   3   3 ...  17  14  11]
 [237 237 238 ... 239 239 240]]
X_prueba: [[203 201 198 ...  25  28  29]
 [ 49  59  66 ...   2  15  36]
 [  7  11  14 ...  26  32  45]
 [ 25  27  29 ... 215 216 216]
 [182 176 156 ...   0   0   0]]
y_entrenamiento: [0 0 0 0 0]
y_prueba: [0 0 1 0 0]


In [26]:
mlp = MLPClassifier(activation='logistic', alpha=0.01, hidden_layer_sizes=(10,10), max_iter=1000, random_state=42, solver='sgd')

start_time = time.time()
mlp.fit(X_entrenamiento, y_entrenamiento)
end_time = time.time()

y_pred = mlp.predict(X_prueba)

print("Predicciones: ", y_pred)
print("Valor real  : ",y_prueba)

accuracy = accuracy_score(y_prueba, y_pred)
print("Precisión: {:.2f}".format(accuracy))

Predicciones:  [0 0 0 ... 0 0 0]
Valor real  :  [0 0 1 ... 0 0 1]
Precisión: 0.83


In [29]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric       Score
0         Recall    0.500000
1      Precision    0.834396
2       Accuracy    0.917198
3             F1    0.454862
4  Training Time  570.593125
